# 課題10
gpsデータの解析

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import folium
import pandas as pd
import numpy as np

In [2]:
fnamer = "log/matsumoto-nagano-100318.log"
# fnamer = "log/nagano-matsumoto-100318.log"

with open(fnamer, "r") as f:
    gpgga_list = []
    for i in f.readlines():
        elements = i.split(",")
        if elements[0] == "$GPGGA":
            gpgga_list.append(elements[1:-1])
gpgga_df = pd.DataFrame(gpgga_list, columns=[
    "time", "lat", "NS", "lon", "EW",
    "qu", "satellites", "harr",
    "elev", "elev_m", "geoid", "geoid_m",
    "DGPS"
])
gpgga_df

,time,lat,NS,lon,EW,qu,satellites,harr,elev,elev_m,geoid,geoid_m,DGPS
0,061959,3614.2227,N,13759.6135,E,1,04,02.2,00613.6,M,037.4,M,00
1,062009,3614.2298,N,13759.5359,E,1,04,02.2,00614.3,M,037.4,M,00
2,062028,3614.2575,N,13759.4495,E,1,04,02.2,00609.0,M,037.4,M,00
3,062043,3614.2540,N,13759.4548,E,1,04,02.2,00604.8,M,037.4,M,00
4,062058,3614.2429,N,13759.4589,E,1,04,02.2,00579.1,M,037.4,M,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,074053,3632.8942,N,13807.8498,E,1,07,01.4,00384.2,M,037.4,M,00
291,074108,3632.9447,N,13807.9225,E,1,09,01.2,00383.0,M,037.4,M,00
292,074123,3632.9938,N,13808.0422,E,1,08,01.2,00382.9,M,037.4,M,00
293,074138,3633.0934,N,13808.1286,E,1,08,01.7,00381.0,M,037.4,M,00


In [3]:
def decode_lat_lon(df, column):
    """
    緯度経度をradianに変換
    """
    tmp = df[column].astype(float)
    df[column] = ((tmp / 100).astype(int) + (tmp % 100) / 60)
#     df[column] = ((tmp / 100).astype(int) + (tmp % 100) / 60).apply(np.deg2rad)

In [4]:
decode_lat_lon(gpgga_df, "lat")
gpgga_df["lat"] = gpgga_df["lat"] * ((gpgga_df.pop("NS") == "N") * 2 - 1)
decode_lat_lon(gpgga_df, "lon")
gpgga_df["lon"] = gpgga_df["lon"] * ((gpgga_df.pop("EW") == "E") * 2 - 1)
gpgga_df

,time,lat,lon,qu,satellites,harr,elev,elev_m,geoid,geoid_m,DGPS
0,061959,36.237045,137.993558,1,04,02.2,00613.6,M,037.4,M,00
1,062009,36.237163,137.992265,1,04,02.2,00614.3,M,037.4,M,00
2,062028,36.237625,137.990825,1,04,02.2,00609.0,M,037.4,M,00
3,062043,36.237567,137.990913,1,04,02.2,00604.8,M,037.4,M,00
4,062058,36.237382,137.990982,1,04,02.2,00579.1,M,037.4,M,00
...,...,...,...,...,...,...,...,...,...,...,...
290,074053,36.548237,138.130830,1,07,01.4,00384.2,M,037.4,M,00
291,074108,36.549078,138.132042,1,09,01.2,00383.0,M,037.4,M,00
292,074123,36.549897,138.134037,1,08,01.2,00382.9,M,037.4,M,00
293,074138,36.551557,138.135477,1,08,01.7,00381.0,M,037.4,M,00


In [5]:
fig = folium.Figure()

center_lat = 36.45
center_lon = 138.0

m = folium.Map([center_lat, center_lon], zoom_start=10).add_to(fig)
for i in gpgga_df.index:
    folium.Marker(location=[gpgga_df["lat"][i], gpgga_df["lon"][i]]).add_to(m)
m

In [6]:
gpgga_df_a = gpgga_df[1:].copy()
gpgga_df_b = gpgga_df[:-1].copy()
gpgga_df_a["lat_prev"] = list(gpgga_df_b["lat"])
gpgga_df_a["lon_prev"] = list(gpgga_df_b["lon"])
gpgga_df_a

,time,lat,lon,qu,satellites,harr,elev,elev_m,geoid,geoid_m,DGPS,lat_prev,lon_prev
1,062009,36.237163,137.992265,1,04,02.2,00614.3,M,037.4,M,00,36.237045,137.993558
2,062028,36.237625,137.990825,1,04,02.2,00609.0,M,037.4,M,00,36.237163,137.992265
3,062043,36.237567,137.990913,1,04,02.2,00604.8,M,037.4,M,00,36.237625,137.990825
4,062058,36.237382,137.990982,1,04,02.2,00579.1,M,037.4,M,00,36.237567,137.990913
5,062113,36.237127,137.989582,1,04,02.2,00580.2,M,037.4,M,00,36.237382,137.990982
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,074053,36.548237,138.130830,1,07,01.4,00384.2,M,037.4,M,00,36.548232,138.130760
291,074108,36.549078,138.132042,1,09,01.2,00383.0,M,037.4,M,00,36.548237,138.130830
292,074123,36.549897,138.134037,1,08,01.2,00382.9,M,037.4,M,00,36.549078,138.132042
293,074138,36.551557,138.135477,1,08,01.7,00381.0,M,037.4,M,00,36.549897,138.134037


In [7]:
Rx = 6378137.0 # 長半径 (m)
Ry = 6356752.314245 # 短半径(m)
E = np.sqrt((Rx ** 2 - Ry ** 2) / Rx ** 2) # 第一離心率

total_dist = 0.0

for i in gpgga_df_a.index:
    row = gpgga_df_a.loc[i, :]
    P = np.deg2rad((row["lat"] + row["lat_prev"]) / 2)
    W = np.sqrt(1 - E ** 2 * np.sin(P) ** 2)
    M = Rx * (1 - E ** 2) / (W ** 3) # 子午線曲率半径
    N = Rx / W # 卯酉線曲率半径
    Dy = np.deg2rad(row["lat"] - row["lat_prev"])
    Dx = np.deg2rad(row["lon"] - row["lon_prev"])
    total_dist += np.sqrt((Dy * M) ** 2 + (Dx * N * np.cos(P)) ** 2)

total_dist / 1000

56.464758387691816

In [9]:
P = np.deg2rad((gpgga_df_a["lat"] + gpgga_df_a["lat_prev"]) / 2)
W = P.apply(lambda x: np.sqrt(1 - E ** 2 * np.sin(x) ** 2))
M = W.apply(lambda x: Rx * (1 - E ** 2) / x ** 3) # 子午線曲率半径
N = W.apply(lambda x: Rx / x) # 卯酉線曲率半径
Dy = (gpgga_df_a["lat"] - gpgga_df_a["lat_prev"]).apply(np.deg2rad)
Dx = (gpgga_df_a["lon"] - gpgga_df_a["lon_prev"]).apply(np.deg2rad)
D = ((Dy * M) ** 2 + (Dx * N * P.apply(np.cos)) ** 2).apply(np.sqrt)

D.sum() / 1000

56.46475838769183